In [1]:
!pip install -q segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00


In [2]:
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import cv2
from torch import nn
from PIL import Image
import segmentation_models_pytorch as smp
from tqdm import tqdm
from zipfile import ZipFile

In [3]:
# -------- Config --------
DATA_DIR = Path('.')  # path to your Carvana dataset
TEST_DIR = DATA_DIR / 'test'  # test folder with .jpg files
MODEL_PATH = '/kaggle/input/carvana-training-pipeline/best_epoch-00.bin'
SUBMISSION_PATH = 'submission.csv'
BATCH_SIZE = 32 * 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_SIZE = (224, 224)
ORIG_SIZE = (1918, 1280)  # (width, height)

In [4]:
%%time
with ZipFile("/kaggle/input/carvana-image-masking-challenge/test.zip") as file:
    file.extractall()

CPU times: user 1min 13s, sys: 17.7 s, total: 1min 30s
Wall time: 2min 6s


In [5]:
# -------- Model --------
def build_model():
    model = smp.Unet(
        encoder_name="efficientnet-b0",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
        activation=None,
    )
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    model.to(DEVICE)
    return model

model = load_model(MODEL_PATH)

config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [6]:
# -------- Image Preprocessing --------
def preprocess_img_batch(paths, input_size=(224, 224)):
    imgs = []
    for path in paths:
        img = cv2.imread(str(path), cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        img = cv2.resize(img, input_size)
        img = np.transpose(img, (2, 0, 1))  # CHW
        imgs.append(img)
    batch = np.stack(imgs)  # (B, 3, H, W)
    return torch.from_numpy(batch).to(DEVICE)

# -------- RLE Encoding --------
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [7]:
# -------- Inference --------
test_paths = sorted(TEST_DIR.glob('*.jpg'))
submission_data = []

for i in tqdm(range(0, len(test_paths), BATCH_SIZE)):
    batch_paths = test_paths[i:i+BATCH_SIZE]
    batch_imgs = preprocess_img_batch(batch_paths, INPUT_SIZE)

    with torch.no_grad():
        preds = model(batch_imgs)
        preds = torch.sigmoid(preds)
        preds = (preds > 0.5).float()

    preds = preds.squeeze(1).cpu().numpy()  # (B, H, W)

    for img_path, pred in zip(batch_paths, preds):
        pred_resized = cv2.resize(pred, ORIG_SIZE)
        pred_bin = (pred_resized > 0.5).astype(np.uint8)
        rle = rle_encode(pred_bin)
        submission_data.append((img_path.name, rle))

100%|██████████| 1564/1564 [54:07<00:00,  2.08s/it]


In [8]:
# -------- Save Submission --------
submission_df = pd.DataFrame(submission_data, columns=["img", "rle_mask"])
submission_df.to_csv(SUBMISSION_PATH, index=False)
print(f"✅ Submission file saved to {SUBMISSION_PATH}")

✅ Submission file saved to submission.csv


In [9]:
submission_df.head()

,img,rle_mask
0,0004d4463b50_01.jpg,614535 276 616451 280 618368 282 620285 284 62...
1,0004d4463b50_02.jpg,614543 268 616460 271 618376 274 620294 275 62...
2,0004d4463b50_03.jpg,614552 285 616468 289 618385 291 620302 292 62...
3,0004d4463b50_04.jpg,603395 10 605312 13 607228 16 609146 17 611064...
4,0004d4463b50_05.jpg,614646 182 614843 2 616562 186 616759 6 618479...


In [10]:
!rm -rf test

In [11]:
ls

__notebook__.ipynb  submission.csv
